<a href="https://colab.research.google.com/github/mrdesautu/Machine-Learning/blob/main/Laberinto_Busqueda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Con programación orientada a objetos implementamos un sistema de búsqueda, el cual utiliza diferentes métodos para encontrar la salida de un laberinto.

With object-oriented programming we implement a search system, which uses different methods to find the way out of a maze.

In [53]:

class Nodo():
    def __init__(self,_estado,_padre):
        self.estado=_estado   #Entendemos por estado (fila,columna)
        self.padre=_padre
        #self.accion=_accion   #Accion es simplemente un texto
                              #que diga que accion se realizo, ejemplo (Arriba,Abajo,Izquierda,Derecha)
                              #No es fundamental para el funcionamiento

class FronteraStack():

    def __init__(self):
        self.frontera=[]
    def __str__(self):
        return "Nodos en la frontera: " + " ".join(
            str(nodo.estado) for nodo in self.frontera)
    def agregar_nodo(self,_nodo):
        #Agregar el nodo pasado por parametro a la frontera
        self.frontera.append(_nodo)
    def quitar_nodo(self):
        #Quitar nodo de la frontera (respetar el tipo de frontera)
        return self.frontera.pop()
    def esta_vacia(self):
        #Comprobar si la frontera está vacia o no
        return len(self.frontera)==0
    def contiene_estado(self,_estado):
        #Comprobar si el estado pasado por parametro ya se encuentra en la frontera
        for nodo in self.frontera:
            if nodo.estado == _estado:
                return True
        return False
'''En esta clase se definirán los siguientes atributos y métodos, para utilizar un modelo depth-first search:
        - Atributos:
            - frontera: Lista que almacenará los nodos en la frontera.
        - Métodos:
            - agregar_nodo: Agregar un nodo a la frontera.
            - quitar_nodo: Quitar un nodo de la frontera.
            - esta_vacia: Comprobar si la frontera está vacía o no.
            - contiene_estado: Comprobar si el estado pasado por parámetro ya se encuentra en la frontera.'''

class FronteraQueue(FronteraStack):
    '''Aplicar herencia con FronteraStack para implementar el método breadth-first search
       La unica diferencia entre ambas es como
       se quitan los nodos, redefiniendo la funcion quitar_nodo
    '''
    def quitar_nodo(self):
        return self.frontera.pop(0)




class Laberinto():
    def  __init__(self,_algoritmo):
        '''Dentro del init podemos ejecutar funciones
           para ir definiendo los atributos de la clase.
           Les dejo lista la parte de leer el laberinto
           del archivo de texto, y la detección del inicio,
           meta y paredes.
        '''
        with open('laberinto.txt','r') as archivo:
            laberinto = archivo.read()     #Con read() leemos todo el archivo y lo guardamos en laberinto
        laberinto=laberinto.splitlines() #Con splitlines() separamos el laberinto en lineas, eliminando el \n
        self.ancho=len(laberinto[0])    #El ancho del laberinto es la cantidad
                                        #de caracteres de la primer linea
                                        #(O de cualquiera suponiendo que todas tienen el mismo ancho)
        self.alto=len(laberinto)        #El alto del laberinto es la cantidad de lineas
        self.paredes=[]                 #Lista de paredes

        for fila in range(self.alto):   #Recorremos todas las filas
            fila_paredes=[]             #Creamos una lista vacia para las paredes de la fila actual
                                        #para cada fila se vuelve a limpiar la lista
            for columna in range(self.ancho): #Recorremos todas las columnas
                if laberinto[fila][columna] == ' ': #Si el caracter es # es una pared
                    fila_paredes.append(False) #Agregamos la pared a la lista de paredes de la fila actual
                elif laberinto[fila][columna] == 'I':   #Si el caracter es I es el inicio
                    self.inicio = (fila,columna)
                    fila_paredes.append(False)         #Guardamos el inicio
                elif laberinto[fila][columna] == 'M':   #Si el caracter es M es la meta
                    self.meta = (fila,columna)
                    fila_paredes.append(False)           #Guardamos la meta
                else:
                    fila_paredes.append(True)
            self.paredes.append(fila_paredes)         #Agregamos la lista de paredes de la fila actual a la lista de paredes
        #De este modo ya tenemos identificadas las paredes, el inicio y la meta
        self.solucion = None
        self.algoritmo = _algoritmo #String en el que pasamos el nombre del algoritmo a utilizar


    def dibujar_laberinto(self):
        # Dibujar el laberinto en su estado actual
        for fila in range(self.alto):
            for columna in range(self.ancho):
                if self.paredes[fila][columna]:
                    print('#', end=' ')
                elif (fila, columna) == self.inicio:
                    print('I', end=' ')
                elif (fila, columna) == self.meta:
                    print('M', end=' ')
                else:
                    print(' ', end=' ')
            print()  # Cambiar de línea después de cada fila

    def dibujar_camino(self, path):
        # Dibujar el laberinto con el camino recorrido
        for fila in range(self.alto):
            for columna in range(self.ancho):
                if self.paredes[fila][columna]:
                    print('#', end=' ')
                elif (fila, columna) == self.inicio:
                    print('I', end=' ')
                elif (fila, columna) == self.meta:
                    print('M', end=' ')
                elif (fila, columna) in path:
                    print('.', end=' ')  # Marcar el camino con puntos
                else:
                    print(' ', end=' ')
            print()  # Cambiar de línea después de cada fila



    def expandir_nodo(self,_nodo):
        '''Dentro de _nodo.estado tenemos la posicion actual del nodo
           Debemos comprobar en todas las direcciones si podemos movernos
           descartando las que sean paredes o esten fuera del laberinto                            (fila_a - 1, colum_a)
           Utilicen el grafico que está en el Notion para guiarse                (fila_a,colum_a-1) (fila_a, colum_a) (fila_a,colum_a+1)
           Devolver una lista de vecinos posibles (nodos hijo)                                     (fila_a + 1, colum_a)
        '''
        #        (0,2)
        #   (1,1)(1,2)(1,3)
        #        (2,2)
        # Nodo(estado:(fila,columna),padre:Nodo)
        fila_a , colum_a = _nodo.estado
        arriba = (fila_a -1, colum_a)
        izquierda = (fila_a , colum_a -1)
        abajo = (fila_a + 1, colum_a)
        derecha = (fila_a , colum_a +1)
        posiciones = [arriba,izquierda,abajo,derecha]
        vecinos = []
        for f,c in posiciones:
            if 0 <= f < self.alto and 0 <= c < self.ancho and not self.paredes[f][c]:
                vecinos.append((f,c))
        return vecinos
    def resolver(self):
        '''
        Acá tienen que implementar el algoritmo de busqueda
        La idea es intentar replicar el pseudocodigo que vimos en clase
        1- Inicializar la frontera con el nodo inicial
        2- Inicializar el conjunto de explorados como vacio
        3- Repetimos:
            3.1- Si la frontera esta vacia, no hay solucion
            3.2- Quitamos un nodo de la frontera
            3.3- Si el nodo contiene un estado que es meta, devolver la solucion
            3.4- Agregar el nodo a explorados
            3.5- Expandir el nodo, agregando los nodos hijos a la frontera
        '''
        if self.algoritmo=='BFS':
            #Crear la frontera que corresponda
            frontera = FronteraQueue()
        elif self.algoritmo=='DFS':
            #Crear la frontera que corresponda
            frontera = FronteraStack()
        #-----------------------------Introducir código--------
        #------------------------------------------------------------------------
        nodo_inicial = Nodo(self.inicio,None)
        frontera.agregar_nodo(nodo_inicial)
        self.nodos_explorados = []
        while True:
            if frontera.esta_vacia():
              raise Exception("No hay solucion")
            nodo_actual = frontera.quitar_nodo()
            if self.algoritmo == 'Greedy':
              # Ordenar la frontera por la distancia Manhattan (heurística)
              frontera.frontera.sort(key=lambda nodo: abs(nodo.estado[0] - self.meta[0]) + abs(nodo.estado[1] - self.meta[1]))

            if nodo_actual.estado == self.meta:
              # Construct the path from the initial node to the goal node
              path = [nodo_actual.estado]
              while nodo_actual.padre:
                  nodo_actual = nodo_actual.padre
                  path.append(nodo_actual.estado)
              path.reverse()
              print("Solución encontrada en", len(path), "pasos:")
              self.dibujar_camino(path)
              return




            self.nodos_explorados.append(nodo_actual.estado)
            vecinos = self.expandir_nodo(nodo_actual)
            for vecino in vecinos:
                if not frontera.contiene_estado(vecino) and vecino not in self.nodos_explorados:
                    nuevo_nodo = Nodo(vecino,nodo_actual)
                    frontera.agregar_nodo(nuevo_nodo)

laberinto = Laberinto("BFS")
laberinto.resolver()

laberinto = Laberinto("DFS")
laberinto.resolver()



Solución encontrada en 49 pasos:
# # # # # # # # # # # # # # # 
# . . . . . . . . . . . . I # 
# . # # # # # # # # # # # # # 
# . #                       # 
# . # # # # # # # # #   #   # 
# . . . . . . . . . . . #   # 
# # # # # # # # #   # . #   # 
#               #   # . #   # 
#     #   # # # # # # . #   # 
#   #           #   # .     # 
#   #   # # #   #   # . # # # 
#   #       #         . . . # 
# # # # #   # # # # # # # . # 
#   . . . . . . . . . . . . # 
# # M # # # # # # # # # # # # 
Solución encontrada en 49 pasos:
# # # # # # # # # # # # # # # 
# . . . . . . . . . . . . I # 
# . # # # # # # # # # # # # # 
# . #                       # 
# . # # # # # # # # #   #   # 
# . . . . . . . . . . . #   # 
# # # # # # # # #   # . #   # 
#               #   # . #   # 
#     #   # # # # # # . #   # 
#   #           #   # .     # 
#   #   # # #   #   # . # # # 
#   #       #         . . . # 
# # # # #   # # # # # # # . # 
#   . . . . . . . . . . . . # 
# # M # # # # # # # # # # # # 


# Gready best-first search
We will use another type of search method, at this time we use an informed search.

In [54]:
class Nodo():
    def __init__(self, estado, padre=None):
        self.estado = estado
        self.padre = padre

class FronteraGreedy():
    def __init__(self):
        self.frontera = []

    def __str__(self):
        return "Nodos en la frontera: " + " ".join(str(nodo.estado) for nodo in self.frontera)

    def agregar_nodo(self, nodo):
        self.frontera.append(nodo)

    def quitar_nodo(self, meta):
        self.frontera.sort(key=lambda nodo: heuristica(nodo, meta))
        return self.frontera.pop(0)

    def esta_vacia(self):
        return len(self.frontera) == 0

    def contiene_estado(self, estado):
        for nodo in self.frontera:
            if nodo.estado == estado:
                return True
        return False

def heuristica(nodo, meta):
    return abs(nodo.estado[0] - meta[0]) + abs(nodo.estado[1] - meta[1])


class Laberinto():
    def __init__(self):
        with open('laberinto.txt', 'r') as archivo:
            laberinto = archivo.read()
        laberinto = laberinto.splitlines()
        self.ancho = len(laberinto[0])
        self.alto = len(laberinto)
        self.paredes = []

        for fila in range(self.alto):
            fila_paredes = []
            for columna in range(self.ancho):
                if laberinto[fila][columna] == ' ':
                    fila_paredes.append(False)
                elif laberinto[fila][columna] == 'I':
                    self.inicio = (fila, columna)
                    fila_paredes.append(False)
                elif laberinto[fila][columna] == 'M':
                    self.meta = (fila, columna)
                    fila_paredes.append(False)
                else:
                    fila_paredes.append(True)
            self.paredes.append(fila_paredes)

        self.solucion = None

    def expandir_nodo(self, nodo):
        fila_a, colum_a = nodo.estado
        arriba = (fila_a - 1, colum_a)
        izquierda = (fila_a, colum_a - 1)
        abajo = (fila_a + 1, colum_a)
        derecha = (fila_a, colum_a + 1)
        posiciones = [arriba, izquierda, abajo, derecha]
        vecinos = []
        for f, c in posiciones:
            if 0 <= f < self.alto and 0 <= c < self.ancho and not self.paredes[f][c]:
                vecinos.append((f, c))
        return vecinos

    def resolver(self):
        frontera = FronteraGreedy()
        nodo_inicial = Nodo(self.inicio)
        frontera.agregar_nodo(nodo_inicial)
        nodos_explorados = []

        while not frontera.esta_vacia():
            nodo_actual = frontera.quitar_nodo(self.meta)

            if nodo_actual.estado == self.meta:
                path = [nodo_actual.estado]
                while nodo_actual.padre:
                    nodo_actual = nodo_actual.padre
                    path.append(nodo_actual.estado)
                path.reverse()
                print("Solución encontrada en", len(path), "pasos:")
                self.dibujar_camino(path)
                return

            nodos_explorados.append(nodo_actual.estado)
            vecinos = self.expandir_nodo(nodo_actual)
            for vecino in vecinos:
                if vecino not in nodos_explorados and not frontera.contiene_estado(vecino):
                    nuevo_nodo = Nodo(vecino, nodo_actual)
                    frontera.agregar_nodo(nuevo_nodo)

        raise Exception("No se encontró solución.")

    def dibujar_laberinto(self):
        for fila in range(self.alto):
            for columna in range(self.ancho):
                if self.paredes[fila][columna]:
                    print('#', end=' ')
                elif (fila, columna) == self.inicio:
                    print('I', end=' ')
                elif (fila, columna) == self.meta:
                    print('M', end=' ')
                else:
                    print(' ', end=' ')
            print()

    def dibujar_camino(self, path):
        for fila in range(self.alto):
            for columna in range(self.ancho):
                if self.paredes[fila][columna]:
                    print('#', end=' ')
                elif (fila, columna) == self.inicio:
                    print('I', end=' ')
                elif (fila, columna) == self.meta:
                    print('M', end=' ')
                elif (fila, columna) in path:
                    print('.', end=' ')
                else:
                    print(' ', end=' ')
            print()


if __name__ == "__main__":
    laberinto = Laberinto()
    laberinto.resolver()

Solución encontrada en 53 pasos:
# # # # # # # # # # # # # # # 
# . . . . . . . . . . . . I # 
# . # # # # # # # # # # # # # 
# . #                       # 
# . # # # # # # # # #   #   # 
# . . . . . . . . . . . #   # 
# # # # # # # # #   # . #   # 
#               #   # . #   # 
#     #   # # # # # # . #   # 
#   # . . . . . #   # .     # 
#   # . # # # . #   # . # # # 
#   # . . . # . . . . .     # 
# # # # # . # # # # # # #   # 
#   . . . .                 # 
# # M # # # # # # # # # # # # 


Podemos notar que el metodo de busqueda tiene un mayor coste en cuanto a los pasos necesarios para encontrar la solución.

#Modelo A Estrella

In [57]:
class FronteraAStar():
    def __init__(self):
        self.frontera = []

    def __str__(self):
        return "Nodos en la frontera: " + " ".join(str(nodo.estado) for nodo in self.frontera)

    def agregar_nodo(self, nodo):
        self.frontera.append(nodo)

    def quitar_nodo(self, meta):
        self.frontera.sort(key=lambda nodo: nodo.costo + heuristica(nodo, meta))
        return self.frontera.pop(0)

    def esta_vacia(self):
        return len(self.frontera) == 0

    def contiene_estado(self, estado):
        for nodo in self.frontera:
            if nodo.estado == estado:
                return True
        return False

class Nodo():
    def __init__(self, estado, padre, costo=0):
        self.estado = estado   # Entendemos por estado (fila, columna)
        self.padre = padre
        self.costo = costo

class Laberinto():
    def __init__(self):
        '''Dentro del init podemos ejecutar funciones
           para ir definiendo los atributos de la clase.
           Les dejo lista la parte de leer el laberinto
           del archivo de texto, y la detección del inicio,
           meta y paredes.
        '''
        with open('laberinto.txt','r') as archivo:
            laberinto = archivo.read()     #Con read() leemos todo el archivo y lo guardamos en laberinto
        laberinto = laberinto.splitlines() #Con splitlines() separamos el laberinto en lineas, eliminando el \n
        self.ancho = len(laberinto[0])    #El ancho del laberinto es la cantidad
                                         #de caracteres de la primer linea
                                         #(O de cualquiera suponiendo que todas tienen el mismo ancho)
        self.alto = len(laberinto)        #El alto del laberinto es la cantidad de lineas
        self.paredes = []                 #Lista de paredes

        for fila in range(self.alto):   #Recorremos todas las filas
            fila_paredes = []             #Creamos una lista vacia para las paredes de la fila actual
                                         #para cada fila se vuelve a limpiar la lista
            for columna in range(self.ancho): #Recorremos todas las columnas
                if laberinto[fila][columna] == ' ': #Si el caracter es ' ' es una pared
                    fila_paredes.append(False) #Agregamos la pared a la lista de paredes de la fila actual
                elif laberinto[fila][columna] == 'I':   #Si el caracter es 'I' es el inicio
                    self.inicio = (fila, columna)
                    fila_paredes.append(False)         #Guardamos el inicio
                elif laberinto[fila][columna] == 'M':   #Si el caracter es 'M' es la meta
                    self.meta = (fila, columna)
                    fila_paredes.append(False)           #Guardamos la meta
                else:
                    fila_paredes.append(True)
            self.paredes.append(fila_paredes)         #Agregamos la lista de paredes de la fila actual a la lista de paredes
        # De este modo ya tenemos identificadas las paredes, el inicio y la meta
        self.solucion = None

    def dibujar_laberinto(self):
        # Dibujar el laberinto en su estado actual
        for fila in range(self.alto):
            for columna in range(self.ancho):
                if self.paredes[fila][columna]:
                    print('#', end=' ')
                elif (fila, columna) == self.inicio:
                    print('I', end=' ')
                elif (fila, columna) == self.meta:
                    print('M', end=' ')
                else:
                    print(' ', end=' ')
            print()  # Cambiar de línea después de cada fila

    def dibujar_camino(self, path):
        # Dibujar el laberinto con el camino recorrido
        for fila in range(self.alto):
            for columna in range(self.ancho):
                if self.paredes[fila][columna]:
                    print('#', end=' ')
                elif (fila, columna) == self.inicio:
                    print('I', end=' ')
                elif (fila, columna) == self.meta:
                    print('M', end=' ')
                elif (fila, columna) in path:
                    print('.', end=' ')  # Marcar el camino con puntos
                else:
                    print(' ', end=' ')
            print()  # Cambiar de línea después de cada fila

    def expandir_nodo(self, _nodo):
        '''Dentro de _nodo.estado tenemos la posicion actual del nodo
           Debemos comprobar en todas las direcciones si podemos movernos
           descartando las que sean paredes o esten fuera del laberinto
           Utilicen el grafico que está en el Notion para guiarse
           Devolver una lista de vecinos posibles (nodos hijo)
        '''
        fila_a, colum_a = _nodo.estado
        arriba = (fila_a - 1, colum_a)
        izquierda = (fila_a, colum_a - 1)
        abajo = (fila_a + 1, colum_a)
        derecha = (fila_a, colum_a + 1)
        posiciones = [arriba, izquierda, abajo, derecha]
        vecinos = []
        for f, c in posiciones:
            if 0 <= f < self.alto and 0 <= c < self.ancho and not self.paredes[f][c]:
                vecinos.append((f, c))
        return vecinos

    def resolver(self):
        '''
        Acá tienen que implementar el algoritmo de búsqueda A*
        '''
        frontera = FronteraAStar()
        nodo_inicial = Nodo(self.inicio, None)
        frontera.agregar_nodo(nodo_inicial)
        self.nodos_explorados = []

        while True:
            if frontera.esta_vacia():
                raise Exception("No hay solucion")
            nodo_actual = frontera.quitar_nodo(self.meta)

            if nodo_actual.estado == self.meta:
                path = [nodo_actual.estado]
                while nodo_actual.padre:
                    nodo_actual = nodo_actual.padre
                    path.append(nodo_actual.estado)
                path.reverse()
                print("Solución encontrada en", len(path), "pasos:")
                self.dibujar_camino(path)
                return

            self.nodos_explorados.append(nodo_actual.estado)
            vecinos = self.expandir_nodo(nodo_actual)

            for vecino in vecinos:
                if vecino not in self.nodos_explorados and not frontera.contiene_estado(vecino):
                    nuevo_costo = nodo_actual.costo + 1  # Se asume costo uniforme (cada paso tiene costo 1)
                    nuevo_nodo = Nodo(vecino, nodo_actual, nuevo_costo)
                    frontera.agregar_nodo(nuevo_nodo)

def heuristica(nodo, meta):
    '''
    Función heurística para el algoritmo A*
    '''
    # Distancia Manhattan desde el nodo actual hasta el nodo objetivo (meta)
    return abs(nodo.estado[0] - meta[0]) + abs(nodo.estado[1] - meta[1])

if __name__ == "__main__":
    laberinto = Laberinto()
    laberinto.resolver()



Solución encontrada en 49 pasos:
# # # # # # # # # # # # # # # 
# . . . . . . . . . . . . I # 
# . # # # # # # # # # # # # # 
# . #                       # 
# . # # # # # # # # #   #   # 
# . . . . . . . . . . . #   # 
# # # # # # # # #   # . #   # 
#               #   # . #   # 
#     #   # # # # # # . #   # 
#   #           #   # .     # 
#   #   # # #   #   # . # # # 
#   #       #         . . . # 
# # # # #   # # # # # # # . # 
#   . . . . . . . . . . . . # 
# # M # # # # # # # # # # # # 
